In [3]:
import os
import re
import numpy as np
import pandas as pd
from scipy.stats import gmean
from sklearn.decomposition import PCA
#Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [12]:
sample_info = pd.read_csv("./data_external/melanoma_data/sample.csv")
sample_info.columns = ['File_Name', 'Case_ID','Sample_ID','Sample_Type']

sample = sample_info.loc[~(
    (sample_info.Sample_Type == 'Solid Tissue Normal') |
    (sample_info.Sample_Type == 'Additional Metastatic')
    )]

print(sample.Sample_Type.value_counts())

Metastatic       367
Primary Tumor    103
Name: Sample_Type, dtype: int64


In [15]:
df_genes = pd.read_csv("./data_generated/ensemble.txt",sep='\t',header=None).iloc[:,0]
df_genes.shape

(60483,)

In [19]:
# create empty list to store expression data from files
data_store = []
# filling the Array files data in datos21 folder
for file in sample.File_Name:
    data_store.append(pd.read_csv(f'./data_external/melanoma_data/datos21/{file}', sep='\t', 
                                  header=None, usecols=[1]))
# transform in a numpy array
data_store = np.array(data_store)
# sum 0.1 to avoid 0 divisions
data_store += 0.1